# Compile and Deploy the TFX Pipeline to KFP

This Notebook helps you to compile the **TFX Pipeline** to a **KFP package**. This will creat an **Argo YAML** file in a **.tar.gz** package. We perform the following steps:
1. Build a custom container image that include our modules
2. Compile TFX Pipeline using CLI
3. Deploy the compiled pipeline to KFP 


## 0. Set compile time variables

In [ ]:
import os

os.environ["PROJECT_ID"]="ksalama-research" # Set your project

os.environ["IMAGE_NAME"]="tfx-image"
os.environ["TAG"]="latest"
os.environ["KFP_TFX_IMAGE"]="gcr.io/{}/{}:{}".format(
    os.environ.get("PROJECT_ID"), 
    os.environ.get("IMAGE_NAME"),
    os.environ.get("TAG"))

os.environ["NAMESPACE"]="kubeflow-pipelines"
os.environ["GCP_REGION"]="europe-west1" # Set your region
os.environ["ARTIFACT_STORE_URI"]="gs://ks-kfp-artifact-store" # Set your GCS Bucket
os.environ["GCS_STAGING_PATH"]=os.environ.get("ARTIFACT_STORE_URI")+"/staging"
os.environ["GKE_CLUSTER_NAME"]="ks-ml-cluster-01" # Set your GKE cluster name
os.environ["GKE_CLUSTER_ZONE"]="europe-west1-b" # Set your GKE cluster zone
os.environ["RUNTIME_VERSION"]="1.15"
os.environ["PYTHON_VERSION"]="3.7"
os.environ["BEAM_RUNNER"]="DirectRunner"

os.environ["PIPELINE_NAME"]="tfx_census_classification"

## 1. Build Container Image

The pipeline uses a custom docker image, which is a derivative of the [tensorflow/tfx:0.21.4](https://hub.docker.com/r/tensorflow/tfx) image, as a runtime execution environment for the pipeline's components. The same image is also used as a a training image used by **AI Platform Training**.

The custom image modifies the base image by adding the `modules` and `raw_schema` folders.


In [ ]:
!gcloud builds submit --tag $KFP_TFX_IMAGE .

## 2. Compile TFX Pipeline using CLI

In [ ]:
#!tfx pipeline compile --help

In [ ]:
!tfx pipeline compile \
    --engine=kubeflow \
    --pipeline_path=ml_pipeline/runner.py 

## 3. Deploy the Compiled Pipeline to KFP

In [ ]:
#!kfp pipeline --help

In [ ]:
%%bash

gcloud container clusters get-credentials ${GKE_CLUSTER_NAME} --zone ${GKE_CLUSTER_ZONE}
export KFP_ENDPOINT=$(kubectl describe configmap inverse-proxy-config -n ${NAMESPACE} | grep "googleusercontent.com")

kfp --namespace=${NAMESPACE} --endpoint=${KFP_ENDPOINT} \
    pipeline upload \
    --pipeline-name=${PIPELINE_NAME} \
    ${PIPELINE_NAME}.tar.gz

## Use the KFP UI to run the deployed pipeline...